In [1]:
import numpy as np
from matplotlib import pyplot as plt
from numpy import random as rnd
import warnings,datetime,os,calendar,csv,time

from scipy.signal import find_peaks
import networkx as nx

import tensorflow as tf
import pandas as pd
import seaborn as sns

from keras.models import Model,Sequential
from keras.layers import Dense,LSTM,Conv2D,Dropout,BatchNormalization,Input,Concatenate,Add,Activation,MaxPooling2D,AveragePooling2D
import keras.backend as K

from sklearn import preprocessing as pp

from sklearn.cluster import KMeans,MeanShift
from sklearn.dummy import DummyClassifier,DummyRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier,AdaBoostRegressor,RandomForestClassifier,RandomForestRegressor
from sklearn.linear_model import BayesianRidge,Lasso,LinearRegression,SGDClassifier,SGDRegressor
from sklearn.mixture import BayesianGaussianMixture,GaussianMixture
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor,RadiusNeighborsClassifier,RadiusNeighborsRegressor,NearestNeighbors
from sklearn.manifold import Isomap
from sklearn.model_selection import KFold,StratifiedKFold,train_test_split
from sklearn.svm import LinearSVC,LinearSVR
from sklearn.neural_network import BernoulliRBM,MLPClassifier,MLPRegressor
from sklearn.decomposition import FactorAnalysis,KernelPCA,PCA,MiniBatchSparsePCA,FastICA
from sklearn.preprocessing import CategoricalEncoder,KBinsDiscretizer,LabelEncoder,MinMaxScaler,OneHotEncoder,StandardScaler

from xgboost import XGBClassifier,XGBRegressor
import gym

import pickle,h5py,json
import urllib
from bs4 import BeautifulSoup

import pygame
from pygame.locals import *

warnings.filterwarnings('ignore')
sns.set()
plt.xkcd()

C:\Users\Jan\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [138]:
BATCHSIZE = 3
NUM_EPOCHS = 50
LR = 0.1

NUM_NODES = 3

In [145]:
tf.reset_default_graph()

sample = tf.placeholder(dtype=tf.float32,shape=[None,NUM_NODES])
C = tf.Variable(tf.random_normal([NUM_NODES,NUM_NODES])) 

act = tf.nn.relu(tf.matmul(sample,C))
E_t = tf.norm(sample)+tf.norm(act)-2*tf.tensordot(sample,act,2)
reg = tf.abs(tf.norm(tf.linalg.diag_part(C)))

loss = tf.add(E_t,reg)

optimize = tf.train.AdamOptimizer(learning_rate=LR).minimize(loss)

In [146]:
D = np.array([[1,0,0],[0,1,0],[0,0,1]]*30)

In [147]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

epoch = 0
while epoch < NUM_EPOCHS:
    DD = D[rnd.permutation(np.arange(D.shape[0]))]
    
    for k in range(DD.shape[0]-BATCHSIZE):
        d = DD[k:k+BATCHSIZE,:]
        free_energy,_ = sess.run([E_t,optimize],feed_dict={
            sample:d.reshape(BATCHSIZE,NUM_NODES)
        })
        print(free_energy)
        print(sess.run(C))
    
    epoch += 1

print(sess.run(C))
sess.close()

2.5687754
[[-0.08697359  0.33449054  0.61507076]
 [-0.77856576 -0.2660586  -0.07324356]
 [-1.4869921  -0.6970488   1.1151998 ]]
0.8184147
[[ 0.00678483  0.24159417  0.52057815]
 [-0.77856576 -0.16714111 -0.07324356]
 [-1.4869921  -0.6970488   1.0635473 ]]
0.9157121
[[ 0.07359402  0.14683987  0.4236359 ]
 [-0.77856576 -0.0718345  -0.07324356]
 [-1.4869921  -0.6970488   1.0386397 ]]
0.58173203
[[ 0.1533088   0.05430038  0.3259836 ]
 [-0.77856576  0.01498898 -0.07324356]
 [-1.4869921  -0.6970488   1.0274615 ]]
0.21328902
[[ 0.2395342  -0.03033598  0.22912252]
 [-0.77856576  0.08638152 -0.07324356]
 [-1.4869921  -0.6970488   1.02443   ]]
-1.3585386
[[ 0.32414904 -0.1032815   0.14083499]
 [-0.77856576  0.13632648 -0.07324356]
 [-1.4869921  -0.6970488   1.0676644 ]]
-2.8246865
[[ 0.39581907 -0.16698526  0.06373294]
 [-0.77856576  0.16387402 -0.07324356]
 [-1.4869921  -0.6970488   1.1304231 ]]
-1.5103526
[[ 0.45652992 -0.22314398 -0.00423719]
 [-0.77856576  0.21778882 -0.07324356]
 [-1.486992

-15.320574
[[ 4.1840663  -0.74542993 -0.63637096]
 [-0.77856576  3.8737516  -0.07324356]
 [-1.4869921  -0.6970488   4.5599093 ]]
-15.679322
[[ 4.2741284  -0.7455342  -0.63649714]
 [-0.77856576  3.9548788  -0.07324356]
 [-1.4869921  -0.6970488   4.558209  ]]
-16.050827
[[ 4.368846   -0.74562865 -0.6366115 ]
 [-0.77856576  4.034591   -0.07324356]
 [-1.4869921  -0.6970488   4.5520854 ]]
-16.433388
[[ 4.4676156  -0.7457143  -0.6367152 ]
 [-0.77856576  4.1130247  -0.07324356]
 [-1.4869921  -0.6970488   4.541998  ]]
-16.929935
[[ 4.5621758  -0.7457919  -0.6368091 ]
 [-0.77856576  4.190328   -0.07324356]
 [-1.4869921  -0.6970488   4.5389047 ]]
-16.642532
[[ 4.6439743  -0.74586225 -0.6368942 ]
 [-0.77856576  4.2753196  -0.07324356]
 [-1.4869921  -0.6970488   4.5421505 ]]
-16.891266
[[ 4.714173   -0.74592596 -0.63697135]
 [-0.77856576  4.366954   -0.07324356]
 [-1.4869921  -0.6970488   4.55126   ]]
-17.394684
[[ 4.7827945  -0.74598366 -0.6370412 ]
 [-0.77856576  4.4643927  -0.07324356]
 [-1.486

-32.000328
[[ 7.9010572  -0.74653417 -0.6377073 ]
 [-0.77856576  7.8277564  -0.07324356]
 [-1.4869921  -0.6970488   8.158715  ]]
-32.1446
[[ 7.973894   -0.74653435 -0.6377075 ]
 [-0.77856576  7.8727956  -0.07324356]
 [-1.4869921  -0.6970488   8.216654  ]]
-32.84421
[[ 8.035729   -0.74653447 -0.63770765]
 [-0.77856576  7.9196415  -0.07324356]
 [-1.4869921  -0.6970488   8.283939  ]]
-32.583668
[[ 8.087615   -0.7465346  -0.6377078 ]
 [-0.77856576  7.977792   -0.07324356]
 [-1.4869921  -0.6970488   8.350277  ]]
-33.000317
[[ 8.140306   -0.7465347  -0.63770795]
 [-0.77856576  8.036411   -0.07324356]
 [-1.4869921  -0.6970488   8.415785  ]]
-33.25176
[[ 8.193744   -0.7465348  -0.63770807]
 [-0.77856576  8.095466   -0.07324356]
 [-1.4869921  -0.6970488   8.480553  ]]
-33.50385
[[ 8.247872   -0.74653494 -0.6377082 ]
 [-0.77856576  8.154925   -0.07324356]
 [-1.4869921  -0.6970488   8.544662  ]]
-33.75657
[[ 8.302642   -0.746535   -0.6377083 ]
 [-0.77856576  8.214762   -0.07324356]
 [-1.4869921  

[[14.529096   -0.7465356  -0.6377091 ]
 [-0.77856576 13.520815   -0.07324356]
 [-1.4869921  -0.6970488  13.745331  ]]
-56.292793
[[14.552249   -0.7465356  -0.6377091 ]
 [-0.77856576 13.595063   -0.07324356]
 [-1.4869921  -0.6970488  13.823044  ]]
-56.938934
[[14.569108   -0.7465356  -0.6377091 ]
 [-0.77856576 13.668018   -0.07324356]
 [-1.4869921  -0.6970488  13.908413  ]]
-57.62835
[[14.580295   -0.7465356  -0.6377091 ]
 [-0.77856576 13.729898   -0.07324356]
 [-1.4869921  -0.6970488  14.009486  ]]
-58.86737
[[14.595828   -0.7465356  -0.6377091 ]
 [-0.77856576 13.781782   -0.07324356]
 [-1.4869921  -0.6970488  14.1157    ]]
-59.19299
[[14.615316   -0.7465356  -0.6377091 ]
 [-0.77856576 13.824644   -0.07324356]
 [-1.4869921  -0.6970488  14.22645   ]]
-60.090153
[[14.647853   -0.7465356  -0.6377091 ]
 [-0.77856576 13.859366   -0.07324356]
 [-1.4869921  -0.6970488  14.332082  ]]
-59.20691
[[14.682668   -0.7465356  -0.6377091 ]
 [-0.77856576 13.896829   -0.07324356]
 [-1.4869921  -0.697048

-74.444855
[[17.745646   -0.7465356  -0.6377091 ]
 [-0.77856576 18.254871   -0.07324356]
 [-1.4869921  -0.6970488  17.719126  ]]
-74.68924
[[17.780094   -0.7465356  -0.6377091 ]
 [-0.77856576 18.33459    -0.07324356]
 [-1.4869921  -0.6970488  17.777325  ]]
-74.9341
[[17.817032   -0.7465356  -0.6377091 ]
 [-0.77856576 18.4119     -0.07324356]
 [-1.4869921  -0.6970488  17.83575   ]]
-76.02583
[[17.846397   -0.7465356  -0.6377091 ]
 [-0.77856576 18.496212   -0.07324356]
 [-1.4869921  -0.6970488  17.89443   ]]
-77.20884
[[17.868935   -0.7465356  -0.6377091 ]
 [-0.77856576 18.595592   -0.07324356]
 [-1.4869921  -0.6970488  17.943325  ]]
-76.59367
[[17.895248   -0.7465356  -0.6377091 ]
 [-0.77856576 18.699533   -0.07324356]
 [-1.4869921  -0.6970488  17.983397  ]]
-75.78132
[[17.934675   -0.7465356  -0.6377091 ]
 [-0.77856576 18.798544   -0.07324356]
 [-1.4869921  -0.6970488  18.015512  ]]
-74.812225
[[17.995045   -0.7465356  -0.6377091 ]
 [-0.77856576 18.883879   -0.07324356]
 [-1.4869921  -

...

-990.79504
[[ 2.3228271e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.7856576e-01  2.3191393e+02 -7.3243558e-02]
 [-1.4869921e+00 -6.9704878e-01  2.3401692e+02]]
-988.06476
[[ 2.3233337e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.7856576e-01  2.3199712e+02 -7.3243558e-02]
 [-1.4869921e+00 -6.9704878e-01  2.3406065e+02]]
-988.8982
[[ 2.3238489e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.7856576e-01  2.3208771e+02 -7.3243558e-02]
 [-1.4869921e+00 -6.9704878e-01  2.3409589e+02]]
-989.22925
[[ 2.3243716e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.7856576e-01  2.3218492e+02 -7.3243558e-02]
 [-1.4869921e+00 -6.9704878e-01  2.3412346e+02]]
-989.5802
[[ 2.3249014e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.7856576e-01  2.3228810e+02 -7.3243558e-02]
 [-1.4869921e+00 -6.9704878e-01  2.3414415e+02]]
-992.29944
[[ 2.3253381e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.7856576e-01  2.3239664e+02 -7.3243558e-02]
 [-1.4869921e+00 -6.9704878e-01  2.3416878e+02]]
-995.1647
[[ 2.3256911e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.785

-1014.45667
[[ 2.3733560e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.7856576e-01  2.3693195e+02 -7.3243558e-02]
 [-1.4869921e+00 -6.9704878e-01  2.4062811e+02]]
-1014.72876
[[ 2.3737976e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.7856576e-01  2.3700847e+02 -7.3243558e-02]
 [-1.4869921e+00 -6.9704878e-01  2.4068457e+02]]
-1015.027
[[ 2.3741548e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.7856576e-01  2.3709306e+02 -7.3243558e-02]
 [-1.4869921e+00 -6.9704878e-01  2.4074126e+02]]
-1015.3484
[[ 2.3744362e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.7856576e-01  2.3718491e+02 -7.3243558e-02]
 [-1.4869921e+00 -6.9704878e-01  2.4079817e+02]]
-1015.6909
[[ 2.3746492e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.7856576e-01  2.3728325e+02 -7.3243558e-02]
 [-1.4869921e+00 -6.9704878e-01  2.4085529e+02]]
-1021.1341
[[ 2.3748007e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.7856576e-01  2.3737770e+02 -7.3243558e-02]
 [-1.4869921e+00 -6.9704878e-01  2.4092255e+02]]
-1016.41675
[[ 2.3748969e+02 -7.4653560e-01 -6.3770908e-01]
 [-

[[ 2.4306400e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.7856576e-01  2.4257559e+02 -7.3243558e-02]
 [-1.4869921e+00 -6.9704878e-01  2.4852049e+02]]
-1034.9578
[[ 2.4318901e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.7856576e-01  2.4260466e+02 -7.3243558e-02]
 [-1.4869921e+00 -6.9704878e-01  2.4854187e+02]]
-1042.9451
[[ 2.4330745e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.7856576e-01  2.4263680e+02 -7.3243558e-02]
 [-1.4869921e+00 -6.9704878e-01  2.4856688e+02]]
-1042.2382
[[ 2.4341013e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.7856576e-01  2.4268164e+02 -7.3243558e-02]
 [-1.4869921e+00 -6.9704878e-01  2.4859517e+02]]
-1042.4062
[[ 2.4349861e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.7856576e-01  2.4273790e+02 -7.3243558e-02]
 [-1.4869921e+00 -6.9704878e-01  2.4862640e+02]]
-1035.3428
[[ 2.4358414e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.7856576e-01  2.4280434e+02 -7.3243558e-02]
 [-1.4869921e+00 -6.9704878e-01  2.4865044e+02]]
-1036.763
[[ 2.4367671e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.7856576e-01 

-1072.9076
[[ 2.4944606e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.7856576e-01  2.4875844e+02 -7.3243558e-02]
 [-1.4869921e+00 -6.9704878e-01  2.5341089e+02]]
-1073.1774
[[ 2.4948318e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.7856576e-01  2.4883005e+02 -7.3243558e-02]
 [-1.4869921e+00 -6.9704878e-01  2.5347913e+02]]
-1073.4735
[[ 2.4951263e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.7856576e-01  2.4890051e+02 -7.3243558e-02]
 [-1.4869921e+00 -6.9704878e-01  2.5355624e+02]]
-1073.7931
[[ 2.4953517e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.7856576e-01  2.4896996e+02 -7.3243558e-02]
 [-1.4869921e+00 -6.9704878e-01  2.5364133e+02]]
-1074.9418
[[ 2.4956139e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.7856576e-01  2.4902846e+02 -7.3243558e-02]
 [-1.4869921e+00 -6.9704878e-01  2.5373357e+02]]
-1068.5454
[[ 2.4959088e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.7856576e-01  2.4908711e+02 -7.3243558e-02]
 [-1.4869921e+00 -6.9704878e-01  2.5382242e+02]]
-1068.7969
[[ 2.4962334e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.

-1090.6796
[[ 2.5449345e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.7856576e-01  2.5412114e+02 -7.3243558e-02]
 [-1.4869921e+00 -6.9704878e-01  2.5944763e+02]]
-1083.3735
[[ 2.5454036e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.7856576e-01  2.5420636e+02 -7.3243558e-02]
 [-1.4869921e+00 -6.9704878e-01  2.5949258e+02]]
-1084.158
[[ 2.5459824e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.7856576e-01  2.5428905e+02 -7.3243558e-02]
 [-1.4869921e+00 -6.9704878e-01  2.5952896e+02]]
[[ 2.5459824e+02 -7.4653560e-01 -6.3770908e-01]
 [-7.7856576e-01  2.5428905e+02 -7.3243558e-02]
 [-1.4869921e+00 -6.9704878e-01  2.5952896e+02]]
